In [2]:
import json
import pandas as pd
import numpy as np
from itertools import combinations
import itertools
import os

# Conteggi Emotion-Neutral

In [3]:
path = "/Users/lucadicola/Tesi Python/Interlayers"

# to store files in a list
neutral_interlayers = []

for (root, dirs, file) in os.walk(path):
    for file_name in file:
        if "neutral" in file_name:
            neutral_interlayers.append(file_name)

for file_name in neutral_interlayers:
    results = []
    #Slipt filename to get emotion_from and emotion_to
    parts = file_name.split("_")
    emotion_from = parts[0]
    emotion_to = parts[1].replace(".json", "")
    # Load data from file
    with open(f"{path}/{file_name}", "r", encoding="utf-8") as f:
        data = json.load(f)
    
    # Collect all unique words
    all_words = {pair["word1"] for pair in data}
    word_list = sorted(all_words)

    # Count occurrences of each word in "word1"
    word_counts = {word: 0 for word in word_list}

    for pair in data:
        word_counts[pair["word1"]] += 1
    
    # Create a DataFrame from the word counts
    df_counts_neutral = pd.DataFrame(list(word_counts.items()), columns=["word", "count"])

    # Save the DataFrame to a CSV file called "counts_neutral_{emotion_from}.csv"
    df_counts_neutral.to_csv(f"Counts Neutral/counts_neutral_{emotion_from}.csv", index=False)

# Conteggio Emo-Opposite

In [4]:
plutchik_opposites = [
    ["joy", "sadness"],
    ["trust", "disgust"],
    ["fear", "anger"],
    ["anticipation", "surprise"]
]

path = "/Users/lucadicola/Tesi Python/Interlayers"

for (root, dirs, file) in os.walk(path):
    for file_name in file:
        parts = file_name.split("_")
        emotion_from = parts[0]
        emotion_to = parts[1].replace(".json", "") 
        if [emotion_from, emotion_to] in plutchik_opposites:
            with open(f"{path}/{file_name}", "r", encoding="utf-8") as f:
                data = json.load(f)
            
            # Collect all unique words
            all_words_1 = {pair["word1"] for pair in data}
            word_list_1 = sorted(all_words_1)

            all_words_2 = {pair["word2"] for pair in data}
            word_list_2 = sorted(all_words_2)

            # Count occurrences of each word in "word1"
            word_counts_1 = {word: 0 for word in word_list_1}
            word_counts_2 = {word: 0 for word in word_list_2}

            for pair in data:
                word_counts_1[pair["word1"]] += 1
                word_counts_2[pair["word2"]] += 1
 
            # Create a DataFrame from the word counts
            df_counts_opposite_1 = pd.DataFrame(list(word_counts_1.items()), columns=["word", "count"])
            # Add emotion column with the value of emotion_from
            df_counts_opposite_1["emotion"] = emotion_from
            df_counts_opposite_2 = pd.DataFrame(list(word_counts_2.items()), columns=["word", "count"])
            # Add emotion column with the value of emotion_to
            df_counts_opposite_2["emotion"] = emotion_to
            # Save the DataFrame to a CSV file called "counts_opposite_{emotion_from}.csv"
            df_counts_opposite_1.to_csv(f"Counts Opposite/counts_opposite_{emotion_from}.csv", index=False)
            df_counts_opposite_2.to_csv(f"Counts Opposite/counts_opposite_{emotion_to}.csv", index=False)

# Conteggio Adiacenti

In [5]:
emotions_adjacent = {
    "joy": ["anticipation", "fear"],
    "trust": ["joy", "fear"],
    "fear": ["trust", "surprise"],
    "surprise": ["fear", "sadness"],
    "sadness": ["surprise", "disgust"],
    "disgust": ["sadness", "anger"],
    "anger": ["disgust", "anticipation"],
    "anticipation": ["anger", "joy"]
}

In [6]:
import os
import json
import pandas as pd

emotions_adjacent = {
    "joy": ["anticipation", "trust"],
    "trust": ["joy", "fear"],
    "fear": ["trust", "surprise"],
    "surprise": ["fear", "sadness"],
    "sadness": ["surprise", "disgust"],
    "disgust": ["sadness", "anger"],
    "anger": ["disgust", "anticipation"],
    "anticipation": ["anger", "joy"]
}

df_counts_adjacent = pd.DataFrame(columns=[
    "word", "emotion", "clockwise_emotion", "count_clockwise", 
    "counterclockwise_emotion", "count_counterclockwise"
])

path = "/Users/lucadicola/Tesi Python/Interlayers"
files = set(os.listdir(path))  # get all filenames once

def load_counts(file_path, key):
    with open(file_path, "r", encoding="utf-8") as f:
        data = json.load(f)
    count = {}
    for pair in data:
        word = pair[key]
        count[word] = count.get(word, 0) + 1
    return count

for emotion, (counterclockwise, clockwise) in emotions_adjacent.items():
    base_data = {"emotion": emotion, "clockwise_emotion": clockwise, "counterclockwise_emotion": counterclockwise}

    cw_file1 = f"{emotion}_{clockwise}.json"
    cw_file2 = f"{clockwise}_{emotion}.json"
    ccw_file1 = f"{emotion}_{counterclockwise}.json"
    ccw_file2 = f"{counterclockwise}_{emotion}.json"

    cw_counts = {}
    ccw_counts = {}

    # Clockwise direction
    if cw_file1 in files:
        cw_counts = load_counts(os.path.join(path, cw_file1), "word1")
    elif cw_file2 in files:
        cw_counts = load_counts(os.path.join(path, cw_file2), "word2")

    # Counterclockwise direction
    if ccw_file1 in files:
        ccw_counts = load_counts(os.path.join(path, ccw_file1), "word1")
    elif ccw_file2 in files:
        ccw_counts = load_counts(os.path.join(path, ccw_file2), "word2")

    # Merge counts
    all_words = set(cw_counts.keys()).union(ccw_counts.keys())
    for word in all_words:
        row = {
            "word": word,
            "emotion": emotion,
            "clockwise_emotion": clockwise,
            "count_clockwise": cw_counts.get(word, 0),
            "counterclockwise_emotion": counterclockwise,
            "count_counterclockwise": ccw_counts.get(word, 0)
        }
        df_counts_adjacent = df_counts_adjacent._append(row, ignore_index=True)

In [7]:
# Save the DataFrame to a CSV file
df_counts_adjacent.to_csv("Counts Adjacent/counts_adjacent.csv", index=False)

# Parole per Valence-Arousal

In [8]:
path = "/Users/lucadicola/Tesi Python/Interlayers"

for (root, dirs, file) in os.walk(path):
    for file_name in file:
        parts = file_name.split("_")
        emotion_from = parts[0]
        emotion_to = parts[1].replace(".json", "") 
        if emotion_from == "neutral" or emotion_to == "neutral":
            continue
        else:
            with open(f"{path}/{file_name}", "r", encoding="utf-8") as f:
                data = json.load(f)
                
                # Collect all unique words
                all_words_1 = {pair["word1"] for pair in data}
                word_list_1 = sorted(all_words_1)

                all_words_2 = {pair["word2"] for pair in data}
                word_list_2 = sorted(all_words_2)

                all_words = all_words_1 & all_words_2
    
                # Create a txt file to store all_words
                with open(f"Interlayer Words/words_{emotion_from}_{emotion_to}.txt", "w") as f:
                    for word in all_words:
                        f.write(f"{word}\n")